In [1]:
import sys
sys.path.append('/home/alexj/hgcal-dev')

In [2]:
from hgcal_dev.visualization.hcal import HCalEvent
import numpy as np
import pandas as pd
import torch
from hgcal_dev.utils.experiment import Experiment
from pathlib import Path
from hgcal_dev.clustering.meanshift import MeanShift
from tqdm import tqdm
from sklearn.cluster import estimate_bandwidth
from sklearn.manifold import TSNE
import plotly
import plotly.express as px
import colorsys
from hgcal_dev.training.criterion import centroid_instance_loss
from sklearn import cluster

In [3]:
run_path = Path('/home/alexj/outputs/wandb/run-20210121_034253-4dpv3phl')
ckpt_path = Path('/home/alexj/outputs/wandb/run-20210121_034253-4dpv3phl/files/hgcal-spvcnn/4dpv3phl/checkpoints/epoch=4-v0.ckpt')
experiment = Experiment(run_path, ckpt_path)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: alexschuy (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [13]:
palette = 'rgb(120,62,62), rgb(94,79,49), rgb(53,69,53), rgb(56,90,107), rgb(80,56,107), rgb(82,42,42), rgb(69,64,53), rgb(92,120,98), rgb(82,97,107), rgb(63,42,82), rgb(120,92,92), rgb(56,49,29), rgb(62,120,81), rgb(63,74,82), rgb(91,73,94), rgb(69,53,53), rgb(94,89,73), rgb(49,94,67), rgb(56,83,107), rgb(120,62,120), rgb(94,55,49), rgb(120,114,92), rgb(73,94,81), rgb(42,63,82), rgb(69,36,69), rgb(56,33,29), rgb(107,100,56), rgb(36,69,51), rgb(29,44,56), rgb(94,49,91), rgb(94,76,73), rgb(82,76,42), rgb(43,56,50), rgb(49,67,94), rgb(56,29,54), rgb(120,78,62), rgb(120,120,62), rgb(92,120,109), rgb(36,49,69), rgb(120,92,116), rgb(69,45,36), rgb(68,69,53), rgb(56,107,90), rgb(92,103,120), rgb(56,43,54), rgb(120,100,92), rgb(91,94,73), rgb(29,56,49), rgb(62,81,120), rgb(82,63,75), rgb(94,64,49), rgb(85,94,49), rgb(62,120,112), rgb(49,55,94), rgb(120,62,97), rgb(94,80,73), rgb(60,69,36), rgb(49,94,88), rgb(29,33,56), rgb(82,42,63), rgb(120,85,62), rgb(112,120,92), rgb(53,69,67), rgb(92,96,120), rgb(56,29,42), rgb(56,40,29), rgb(45,56,29), rgb(36,69,67), rgb(73,76,94), rgb(107,56,73), rgb(94,70,49), rgb(51,56,43), rgb(73,94,93), rgb(53,54,69), rgb(69,36,45), rgb(120,105,92), rgb(81,120,62), rgb(56,104,107), rgb(36,36,69), rgb(120,92,100), rgb(56,49,43), rgb(45,69,36), rgb(92,118,120), rgb(59,56,107), rgb(94,73,78), rgb(120,93,62), rgb(78,94,73), rgb(43,54,56), rgb(91,82,107), rgb(94,49,55), rgb(69,53,36), rgb(55,94,49), rgb(49,85,94), rgb(40,29,56), rgb(56,29,31), rgb(120,101,62), rgb(29,56,29), rgb(36,62,69), rgb(70,63,82)'.split(', ')


In [4]:
events = experiment.get_events('train', n=10)

100%|██████████| 10/10 [00:00<00:00, 13.09it/s]


In [16]:
events[0].input_event['instance'] = events[0].input_event['labels_i'].astype(str)

In [17]:
px.scatter(events[0].input_event, x='phi', y='eta', color='instance', color_discrete_sequence=palette)

In [47]:
clusterer = cluster.KMeans(n_clusters=64)
pred_instance_labels = clusterer.fit_predict(events[0].input_event[['eta', 'phi']], sample_weight=events[0].input_event['energy'])
events[0].input_event['pred_instance'] = pred_instance_labels.astype(str)
events[0].pq(pred_instance_labels)

(array([0.84877502, 0.71355828]),
 array([0.09375   , 0.42477876]),
 0.19133853044365323)

In [48]:
clusterer = MeanShift(use_gpu=False, bandwidth=0.225)
pred_instance_labels = clusterer.cluster(events[0].input_event[['eta', 'phi']])
events[0].input_event['pred_instance'] = pred_instance_labels.astype(str)
events[0].pq(pred_instance_labels)

(array([0.93068013, 0.82967876]),
 array([0.11764706, 0.51315789]),
 0.26762399332842957)

In [39]:
px.scatter(events[0].input_event, x='phi', y='eta', color='pred_instance', color_discrete_sequence=palette)


In [10]:
events[0].input_event

,fUniqueID,fBits,energy,timeFalling,time,x,y,z,eta,phi,RHClusterDist,labels_s,particleMatch,nRHClusterMatches,labels_i
0,0,50331648,0.259219,-120.0,-12.500000,180.880447,7.897420,-7.878280,-0.0435,0.043633,4.436671e-01,0,0,0,45
1,0,50331648,0.182575,-120.0,12.500000,172.673096,54.443596,-7.878280,-0.0435,0.305432,5.577382e-01,0,0,0,45
2,0,50331648,0.000000,-120.0,-9999.000000,167.270950,69.285896,-7.878280,0.0000,0.000000,4.804853e-01,0,0,0,45
3,0,50331648,0.413096,-120.0,12.500000,152.698349,97.279587,-7.878279,-0.0435,0.567232,7.499210e-01,0,0,0,45
4,0,50331648,0.000000,-120.0,-9999.000000,7.897408,180.880447,-7.878280,0.0000,0.000000,4.804853e-01,0,0,0,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0,50331648,1.869458,-100.0,12.500000,56.401131,-26.300301,-522.797974,-2.8250,-0.436333,8.339614e-02,1,0,0,33
1071,0,50331648,1.250082,-100.0,3.840148,77.473808,54.247746,522.797974,2.4110,0.610865,4.036113e-08,1,0,0,41
1072,0,50331648,0.832250,-100.0,1.828216,56.401154,26.300289,522.797974,2.8250,0.436332,1.435775e-01,1,0,0,18
1073,0,50331648,3.115420,4.5,4.953614,50.977299,35.694687,522.797974,2.8250,0.610865,3.118467e-02,1,0,0,18
